In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import numpy as np
from pathlib import Path

In [2]:
from ml.predictor import BasicBlockPredictor
from ml.online_trainer import OnlineLearner
from ml.dataset import analyze_sequence_stats
from bb_toolkit import BasicBlockTokenizer, TraceData

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
def load_all_traces(traces_dir: os.PathLike) -> list[TraceData]:
    """Load all .trace files from directory, sorted by creation time."""
    files = sorted(Path(traces_dir).glob("*.trace"), key=lambda f: f.stat().st_ctime)
    return [TraceData.from_binary_file(file) for file in files]

In [5]:
trace_dir = Path("./collection/traces")
traces = load_all_traces(trace_dir)

len(traces)

315

In [6]:
seqs = [trace.sequence().blocks() for trace in traces]
blocks = [item for sublist in seqs for item in sublist]
len(blocks)

1525117

In [ ]:
import random

random.seed(100)
random.shuffle(traces)

# partition
initial_traces = traces[0:250]
exp_traces = traces[250:]

len(initial_traces), len(exp_traces)

In [ ]:
# create predictor
tokenizer_path = Path("./run/tokenizer.bin")
model_path = Path("./run/model.pt")

predictor = BasicBlockPredictor(
    tokenizer_path = tokenizer_path,
    model_path = model_path,
    device=device
)

In [ ]:
# tokenizer initial traces
print(f"initial len: {len(predictor.tokenizer)}")

initial_sequences = [predictor.tokenizer.process_trace(trace) for trace in initial_traces]
len(predictor.tokenizer)

In [ ]:
# example token
print(predictor.tokenizer.get_block(15000))

In [ ]:
# sequence stats
analyze_sequence_stats(initial_sequences)

In [ ]:
initial_training_results = predictor.learner.initial_training(
    trace_sequences=initial_sequences,
    epochs=3,
    # test_split = None
    test_split=0.025
)

predictor.save_model(model_path, tokenizer_path)

initial_training_results

In [ ]:
predictor.get_model_stats()

In [ ]:
def test_accuracy(predictor, test_sequence, offset: int = 0, num_tests=50, context_length=64, top_k=5):
      correct_top1 = 0
      correct_topk = 0
      total_predictions = 0

      for i in range(num_tests):
          start_pos = offset + i
          context = test_sequence[start_pos:start_pos + context_length]
          actual_next = test_sequence[start_pos + context_length]

          try:
              predictions = predictor.predict_next_block_from_sequence(context, top_k=top_k)
              if predictions:
                  predicted_tokens = [pred[0] for pred in predictions]

                  # Top-1 accuracy
                  if predicted_tokens[0] == actual_next:
                      correct_top1 += 1

                  # Top-K accuracy
                  if actual_next in predicted_tokens:
                      correct_topk += 1

                  total_predictions += 1
          except Exception as e:
              print(f"Error at position {start_pos}: {e}")
              continue

      if total_predictions == 0:
          return {'error': 'No successful predictions made'}

      results = {
          'top1_accuracy': correct_top1 / total_predictions,
          'topk_accuracy': correct_topk / total_predictions,
          'total_tests': total_predictions,
          'context_length': context_length,
          'top_k': top_k
      }

      print(f"Results ({total_predictions} tests):")
      print(f"  Top-1 Accuracy: {results['top1_accuracy']:.4f} ({results['top1_accuracy']*100:.1f}%)")
      print(f"  Top-{top_k} Accuracy: {results['topk_accuracy']:.4f} ({results['topk_accuracy']*100:.1f}%)")

      return results

In [ ]:
test_trace, *exp_traces = exp_traces

In [ ]:
test_blocks = test_trace.sequence().symbolized_blocks()

In [ ]:
test_accuracy(predictor, test_blocks, offset=150)

In [ ]:
predictor.clear_cache()

In [ ]:
predictor.predict_next_block_from_sequence(test_blocks[1000:2000], top_k=1)

In [ ]:
print(test_blocks[2001])

In [ ]:
print(predictor.tokenizer.get_block(602))

In [ ]:
# create predictor
tokenizer_path = Path("./run/tokenizer.bin")
model_path = Path("./run/model.pt")

predictor = BasicBlockPredictor(
    tokenizer_path = tokenizer_path,
    model_path = model_path,
    device=device
)